# Tool Calling with HelpingAI 🛠️

This notebook demonstrates how to use **function/tool calling** with HelpingAI, providing the same capabilities as OpenAI's tool calling but with HelpingAI's advanced models.

## What You'll Learn
- How to define tools/functions for the AI to use
- Basic tool calling examples
- Advanced multi-step tool calling
- Error handling and validation
- Best practices for tool definitions
- Integration with external APIs and services

## 🔧 Setup

First, let's install the required packages and set up our environment.

In [ ]:
# Install required packages
%pip install -U HelpingAI 

In [ ]:
# Import required libraries
import os
import json
import requests
from datetime import datetime
from HelpingAI import HAI

# Set your API key (replace with your actual key)
# You can get your API key from: https://helpingai.co/dashboard
os.environ["HAI_API_KEY"] = "hl-************************"

# Initialize the client
hai = HAI()

print("✅ HelpingAI client initialized successfully!")
print(f"📅 Current time: {datetime.now()}")

✅ HelpingAI client initialized successfully!
📅 Current time: 2025-07-17 13:33:14.858532


## 🔍 Understanding Tool Calling

Tool calling allows AI models to:
- Call external functions and APIs
- Perform calculations and data processing
- Access real-time information
- Interact with databases and services

The process involves:
1. **Define tools** - Specify functions the AI can call
2. **Send request** - Include tools in your chat completion
3. **Handle tool calls** - Execute the functions the AI requests
4. **Continue conversation** - Send results back to the AI

## 📐 Example 1: Basic Calculator Tool

Let's start with a simple calculator tool that can perform basic mathematical operations.

In [3]:
# Define our calculator functions
def calculate(operation, a, b):
    """Perform basic mathematical operations"""
    operations = {
        "add": lambda x, y: x + y,
        "subtract": lambda x, y: x - y,
        "multiply": lambda x, y: x * y,
        "divide": lambda x, y: x / y if y != 0 else "Error: Division by zero"
    }
    
    if operation in operations:
        result = operations[operation](a, b)
        return {"result": result, "operation": f"{a} {operation} {b} = {result}"}
    else:
        return {"error": f"Unknown operation: {operation}"}

# Define the tool specification for HelpingAI
calculator_tool = {
    "type": "function",
    "function": {
        "name": "calculate",
        "description": "Perform basic mathematical operations: add, subtract, multiply, divide",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "The mathematical operation to perform"
                },
                "a": {
                    "type": "number",
                    "description": "The first number"
                },
                "b": {
                    "type": "number",
                    "description": "The second number"
                }
            },
            "required": ["operation", "a", "b"]
        }
    }
}

print("🧮 Calculator tool defined successfully!")
print(json.dumps(calculator_tool, indent=2))

🧮 Calculator tool defined successfully!
{
  "type": "function",
  "function": {
    "name": "calculate",
    "description": "Perform basic mathematical operations: add, subtract, multiply, divide",
    "parameters": {
      "type": "object",
      "properties": {
        "operation": {
          "type": "string",
          "enum": [
            "add",
            "subtract",
            "multiply",
            "divide"
          ],
          "description": "The mathematical operation to perform"
        },
        "a": {
          "type": "number",
          "description": "The first number"
        },
        "b": {
          "type": "number",
          "description": "The second number"
        }
      },
      "required": [
        "operation",
        "a",
        "b"
      ]
    }
  }
}


In [4]:
# Test the calculator tool with HelpingAI
messages = [
    {
        "role": "user",
        "content": "What is 15 multiplied by 7? Also, what's 100 divided by 4?"
    }
]

# Make the initial request with tools
response = hai.chat.completions.create(
    model="Dhanishtha-2.0-preview",  # Use a capable model
    messages=messages,
    tools=[calculator_tool],
    tool_choice="auto"  # Let the AI decide when to use tools
)

print("🤖 AI Response:")
print(f"Content: {response.choices[0].message.content}")
print(f"\n🔧 Tool calls: {len(response.choices[0].message.tool_calls) if response.choices[0].message.tool_calls else 0}")

# Process any tool calls
if response.choices[0].message.tool_calls:
    for tool_call in response.choices[0].message.tool_calls:
        print(f"\n📞 Tool Call ID: {tool_call.id}")
        print(f"🔧 Function: {tool_call.function.name}")
        print(f"📝 Arguments: {tool_call.function.arguments}")

🤖 AI Response:
Content: <think>
This is a math problem with two parts. The user wants to know:
1. 15 multiplied by 7
2. 100 divided by 4

I need to use the calculate function for both operations. For the first part, operation is \"multiply\" with a=15 and b=7. For the second part, operation is \"divide\" with a=100 and b=4.
</think>

<think>
Let me solve these step by step:
- For 15 × 7: This equals 105
- For 100 ÷ 4: This equals 25

I'll need to make two separate function calls to get both answers, then present them clearly to the user.
</think>



🔧 Tool calls: 2

📞 Tool Call ID: chatcmpl-tool-195b5ded923c4c72a519a1301f6de847
🔧 Function: calculate
📝 Arguments: {"operation": "multiply", "a": 15, "b": 7}

📞 Tool Call ID: chatcmpl-tool-5ee06a4760e14310b9efbd032917f87e
🔧 Function: calculate
📝 Arguments: {"operation": "divide", "a": 100, "b": 4}


In [5]:
# Execute the tool calls and continue the conversation
if response.choices[0].message.tool_calls:
    # Add the assistant's message to the conversation
    messages.append(response.choices[0].message)
    
    # Execute each tool call
    for tool_call in response.choices[0].message.tool_calls:
        if tool_call.function.name == "calculate":
            # Parse the arguments
            args = json.loads(tool_call.function.arguments)
            
            # Execute the function
            result = calculate(args["operation"], args["a"], args["b"])
            
            print(f"🧮 Executing: {args['a']} {args['operation']} {args['b']}")
            print(f"📊 Result: {result}")
            
            # Add the tool result to the conversation
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })
    
    # Get the final response from the AI
    final_response = hai.chat.completions.create(
        model="Dhanishtha-2.0-preview",
        messages=messages
    )
    
    print("\n🎯 Final AI Response:")
    print(final_response.choices[0].message.content)
else:
    print("ℹ️ No tool calls were made in this response.")

🧮 Executing: 15 multiply 7
📊 Result: {'result': 105, 'operation': '15 multiply 7 = 105'}
🧮 Executing: 100 divide 4
📊 Result: {'result': 25.0, 'operation': '100 divide 4 = 25.0'}

🎯 Final AI Response:
Answer: 15 multiplied by 7 equals 105, and 100 divided by 4 equals 25.


## 🌐 Example 2: Weather Information Tool

Let's create a more practical tool that fetches real-time weather information.

In [6]:
# Define weather tool function
def get_weather(city, country_code=None):
    """Get current weather information for a city"""
    try:
        # Using OpenWeatherMap API (free tier)
        # You'll need to get a free API key from https://openweathermap.org/api
        api_key = "your_openweather_api_key"  # Replace with actual key
        
        # For demo purposes, we'll return mock data
        # In real implementation, uncomment the API call below
        
        # location = f"{city},{country_code}" if country_code else city
        # url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
        # response = requests.get(url)
        # data = response.json()
        
        # Mock weather data for demonstration
        mock_weather = {
            "city": city,
            "country": country_code or "Unknown",
            "temperature": 22.5,
            "feels_like": 24.1,
            "humidity": 65,
            "description": "partly cloudy",
            "wind_speed": 3.2,
            "timestamp": datetime.now().isoformat()
        }
        
        return mock_weather
        
    except Exception as e:
        return {"error": f"Could not fetch weather data: {str(e)}"}

# Define the weather tool specification
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather information for a specific city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city"
                },
                "country_code": {
                    "type": "string",
                    "description": "Optional 2-letter country code (e.g., 'US', 'GB', 'IN')"
                }
            },
            "required": ["city"]
        }
    }
}

print("🌤️ Weather tool defined successfully!")

🌤️ Weather tool defined successfully!


In [7]:
# Test the weather tool
def handle_tool_calling_conversation(user_message, tools, available_functions):
    """Helper function to handle complete tool calling conversations"""
    messages = [{"role": "user", "content": user_message}]
    
    # Initial request
    response = hai.chat.completions.create(
        model="Dhanishtha-2.0-preview",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    print(f"🤖 Initial AI Response: {response.choices[0].message.content}")
    
    # Check for tool calls
    if response.choices[0].message.tool_calls:
        # Add assistant message
        messages.append(response.choices[0].message)
        
        # Execute tool calls
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            print(f"\n🔧 Executing tool: {function_name}")
            
            if function_name in available_functions:
                args = json.loads(tool_call.function.arguments)
                print(f"📝 Arguments: {args}")
                
                # Execute the function
                result = available_functions[function_name](**args)
                print(f"📊 Result: {result}")
                
                # Add tool result to messages
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result)
                })
        
        # Get final response
        final_response = hai.chat.completions.create(
            model="Dhanishtha-2.0-preview",
            messages=messages
        )
        
        print("\n🎯 Final AI Response:")
        print(final_response.choices[0].message.content)
        
        return messages, final_response
    
    return messages, response

# Test weather tool
messages, response = handle_tool_calling_conversation(
    "What's the weather like in Tokyo and London? Please compare them.",
    [weather_tool],
    {"get_weather": get_weather}
)

🤖 Initial AI Response: <think>
This is a weather comparison question for Tokyo and London. I need to call the get_weather function for both cities. For Tokyo, I'll need to call it once, and for London, I may need to specify the country code (probably GB for United Kingdom). I'll make two separate function calls to get the weather data for both cities.
</think>

I'll check the current weather conditions for both Tokyo and London for you.

<think>
Let me structure this properly with the required tool_call format. I need to make two separate function calls - one for Tokyo and one for London with the appropriate country code.
</think>



🔧 Executing tool: get_weather
📝 Arguments: {'city': 'Tokyo'}
📊 Result: {'city': 'Tokyo', 'country': 'Unknown', 'temperature': 22.5, 'feels_like': 24.1, 'humidity': 65, 'description': 'partly cloudy', 'wind_speed': 3.2, 'timestamp': '2025-07-17T13:33:26.817848'}

🔧 Executing tool: get_weather
📝 Arguments: {'city': 'London', 'country_code': 'GB'}
📊 Result: {

## 🔗 Example 3: Multi-Tool Environment

Let's combine multiple tools and see how the AI can intelligently choose which tools to use.

In [8]:
# Define additional tools
def get_time_info(timezone=None):
    """Get current time information"""
    from datetime import datetime
    import pytz
    
    if timezone:
        try:
            tz = pytz.timezone(timezone)
            current_time = datetime.now(tz)
        except:
            current_time = datetime.now()
            timezone = "UTC (fallback)"
    else:
        current_time = datetime.now()
        timezone = "Local"
    
    return {
        "current_time": current_time.strftime("%Y-%m-%d %H:%M:%S"),
        "timezone": timezone,
        "day_of_week": current_time.strftime("%A"),
        "timestamp": current_time.timestamp()
    }

def text_analyzer(text, analysis_type="sentiment"):
    """Analyze text for various properties"""
    analyses = {
        "sentiment": {
            "sentiment": "positive" if "good" in text.lower() or "great" in text.lower() else "neutral",
            "confidence": 0.85
        },
        "length": {
            "character_count": len(text),
            "word_count": len(text.split()),
            "sentence_count": text.count('.') + text.count('!') + text.count('?')
        },
        "keywords": {
            "keywords": [word for word in text.lower().split() if len(word) > 4],
            "uppercase_words": [word for word in text.split() if word.isupper()]
        }
    }
    
    return analyses.get(analysis_type, {"error": "Unknown analysis type"})

# Define tool specifications
time_tool = {
    "type": "function",
    "function": {
        "name": "get_time_info",
        "description": "Get current time and date information",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "Timezone (e.g., 'US/Eastern', 'Europe/London', 'Asia/Tokyo')"
                }
            }
        }
    }
}

text_analysis_tool = {
    "type": "function",
    "function": {
        "name": "text_analyzer",
        "description": "Analyze text for sentiment, length, or keywords",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "The text to analyze"
                },
                "analysis_type": {
                    "type": "string",
                    "enum": ["sentiment", "length", "keywords"],
                    "description": "Type of analysis to perform"
                }
            },
            "required": ["text"]
        }
    }
}

# All available tools and functions
all_tools = [calculator_tool, weather_tool, time_tool, text_analysis_tool]
all_functions = {
    "calculate": calculate,
    "get_weather": get_weather,
    "get_time_info": get_time_info,
    "text_analyzer": text_analyzer
}

print("🛠️ Multi-tool environment ready!")
print(f"📊 Available tools: {len(all_tools)}")

🛠️ Multi-tool environment ready!
📊 Available tools: 4


In [9]:
# Test multi-tool conversation
complex_query = """
Hi! I need help with several things:
1. What time is it in New York right now?
2. Calculate the tip for a $85.50 dinner bill (20% tip)
3. Analyze this text for sentiment: "I'm having a great day and feeling wonderful!"
4. What's the weather like in New York?

Please help me with all of these tasks!
"""

print("🚀 Processing complex multi-tool query...\n")

messages, response = handle_tool_calling_conversation(
    complex_query,
    all_tools,
    all_functions
)

🚀 Processing complex multi-tool query...

🤖 Initial AI Response: <think>
This user has asked for multiple tasks in one request. I need to address each one systematically. Let me identify what functions I need to use:

1. Time in New York - I'll need get_time_info with timezone "America/New_York"
2. Tip calculation - I'll need calculate with operation "multiply", a=85.50, b=0.20
3. Text analysis - I'll need text_analyzer with the text and analysis_type "sentiment"
4. Weather in New York - I'll need get_weather with city "New York"

I should organize my response to address all these clearly.
</think>

I'm happy to help with all your questions! Let me work through each one:

1. **Current time in New York**: I'll check what time it is in the Eastern Time Zone.

2. **Tip calculation**: I'll multiply $85.50 by 20% to find the tip amount.

3. **Text analysis**: I'll analyze \"I'm having a great day and feeling wonderful!\" for sentiment.

4. **Weather in New York**: I'll provide you with the 

## 🎛️ Example 4: Advanced Tool Usage with Streaming

Let's explore how tool calling works with streaming responses for real-time interaction.

In [10]:
# Streaming tool calling example
def handle_streaming_tool_calls(user_message, tools, available_functions):
    """Handle tool calling with streaming responses"""
    messages = [{"role": "user", "content": user_message}]
    
    print("🌊 Starting streaming response...\n")
    
    # Create streaming request
    stream = hai.chat.completions.create(
        model="Dhanishtha-2.0-preview",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        stream=True
    )
    
    # Collect the streamed response
    collected_content = ""
    collected_tool_calls = []
    
    for chunk in stream:
        if chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            print(content, end="", flush=True)
            collected_content += content
        
        if chunk.choices[0].delta.tool_calls:
            for tool_call_delta in chunk.choices[0].delta.tool_calls:
                # Handle tool call streaming (simplified)
                collected_tool_calls.append(tool_call_delta)
    
    print("\n\n📝 Streaming completed!")
    
    # If we collected tool calls, execute them
    if collected_tool_calls:
        print("🔧 Processing tool calls...")
        # In a real implementation, you'd need to reconstruct full tool calls
        # from the streamed deltas. For simplicity, we'll use the non-streaming approach
        return handle_tool_calling_conversation(user_message, tools, available_functions)
    
    return messages, None

# Test streaming with tool calls
streaming_query = "Calculate 25% of 240, and tell me what time it is in Tokyo."

messages, response = handle_streaming_tool_calls(
    streaming_query,
    [calculator_tool, time_tool],
    {"calculate": calculate, "get_time_info": get_time_info}
)

🌊 Starting streaming response...

<think>
First, I need to break down the user's request. They want two things: (1) to calculate 25% of 240, and (2) to find out the current time in Tokyo. The first part is a math problem, so I'll use the `calculate` function. The second part is about getting time in a specific timezone, so I'll use the `get_time_info` function.
</think>

<think>
For the calculation, 25% of 240 means multiplying 240 by 0.25, which is the same as dividing 240 by 4. For the time, I need to specify the timezone as "Asia/Tokyo" in the function call. I should make sure to format the response clearly, showing both results.
</think>

To calculate 25% of 240, I'll multiply 240 by 0.25.

240 × 0.25 = 60

Now, let me check the current time in Tokyo for you.



📝 Streaming completed!


## 🛡️ Example 5: Error Handling and Validation

Let's implement robust error handling for tool calling scenarios.

In [11]:
def safe_tool_execution(tool_call, available_functions):
    """Safely execute a tool call with comprehensive error handling"""
    try:
        function_name = tool_call.function.name
        print(f"🔧 Executing: {function_name}")
        
        # Check if function exists
        if function_name not in available_functions:
            return {
                "error": f"Function '{function_name}' not found",
                "available_functions": list(available_functions.keys())
            }
        
        # Parse arguments safely
        try:
            args = json.loads(tool_call.function.arguments)
            print(f"📝 Arguments: {args}")
        except json.JSONDecodeError as e:
            return {
                "error": f"Invalid JSON arguments: {str(e)}",
                "raw_arguments": tool_call.function.arguments
            }
        
        # Execute function with timeout
        try:
            result = available_functions[function_name](**args)
            print(f"✅ Success: {result}")
            return result
        except TypeError as e:
            return {
                "error": f"Invalid arguments for {function_name}: {str(e)}",
                "provided_args": args
            }
        except Exception as e:
            return {
                "error": f"Execution error in {function_name}: {str(e)}",
                "error_type": type(e).__name__
            }
            
    except Exception as e:
        return {
            "error": f"Unexpected error: {str(e)}",
            "error_type": type(e).__name__
        }

def robust_tool_calling_conversation(user_message, tools, available_functions, max_iterations=5):
    """Handle tool calling with robust error handling and iteration limits"""
    messages = [{"role": "user", "content": user_message}]
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        print(f"\n🔄 Iteration {iteration}")
        
        try:
            response = hai.chat.completions.create(
                model="Dhanishtha-2.0-preview",
                messages=messages,
                tools=tools,
                tool_choice="auto"
            )
            
            print(f"🤖 AI Response: {response.choices[0].message.content}")
            
            # Check for tool calls
            if response.choices[0].message.tool_calls:
                # Add assistant message
                messages.append(response.choices[0].message)
                
                # Execute each tool call safely
                for tool_call in response.choices[0].message.tool_calls:
                    result = safe_tool_execution(tool_call, available_functions)
                    
                    # Add tool result
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(result, indent=2)
                    })
                
                # Continue to next iteration for final response
                continue
            else:
                # No more tool calls, we're done
                print("\n🎯 Conversation completed successfully!")
                return messages, response
                
        except Exception as e:
            print(f"❌ Error in iteration {iteration}: {str(e)}")
            messages.append({
                "role": "system",
                "content": f"Error occurred: {str(e)}. Please try a different approach."
            })
    
    print(f"⚠️ Maximum iterations ({max_iterations}) reached")
    return messages, None

# Test error handling
error_test_query = "Calculate the division of 10 by 0, and also calculate 50 plus abc (which should cause an error)."

print("🧪 Testing error handling...")
messages, response = robust_tool_calling_conversation(
    error_test_query,
    [calculator_tool],
    {"calculate": calculate}
)

🧪 Testing error handling...

🔄 Iteration 1
🤖 AI Response: <think>
Let me analyze this step by step. The user wants me to:
1. Calculate 10 divided by 0
2. Calculate 50 plus "abc" (which should cause an error)

For the first calculation, division by 0 is mathematically undefined and will result in an error. For the second calculation, adding a number (50) with a string ("abc") would also cause a type error since we can't add numbers and strings directly.
</think>

I'll approach this calculation methodically.

<think>
Looking at the function provided, I need to use the `calculate` function for both operations. However, there are some important issues:
1. Division by 0 is mathematically undefined and will cause an error
2. Adding a number (50) with a string ("abc") will cause a type error

I should call the function twice - once for each operation - and then explain the results.
</think>

For the first calculation, I'll use the divide operation with a=10 and b=0:

🔧 Executing: calculate
📝 

## 🎨 Example 6: Custom Tools for Specific Use Cases

Let's create some specialized tools for common business scenarios.

In [12]:
# Business-specific tools
def currency_converter(amount, from_currency, to_currency):
    """Convert currency (mock implementation)"""
    # Mock exchange rates
    rates = {
        "USD": {"EUR": 0.85, "GBP": 0.73, "JPY": 110.0, "INR": 74.5},
        "EUR": {"USD": 1.18, "GBP": 0.86, "JPY": 129.5, "INR": 87.8},
        "GBP": {"USD": 1.37, "EUR": 1.16, "JPY": 150.8, "INR": 102.1}
    }
    
    if from_currency == to_currency:
        return {"amount": amount, "from": from_currency, "to": to_currency, "rate": 1.0}
    
    if from_currency in rates and to_currency in rates[from_currency]:
        rate = rates[from_currency][to_currency]
        converted = round(amount * rate, 2)
        return {
            "original_amount": amount,
            "converted_amount": converted,
            "from_currency": from_currency,
            "to_currency": to_currency,
            "exchange_rate": rate,
            "timestamp": datetime.now().isoformat()
        }
    
    return {"error": f"Conversion from {from_currency} to {to_currency} not supported"}

def schedule_manager(action, date=None, time=None, title=None, description=None):
    """Manage calendar events (mock implementation)"""
    if action == "create":
        event = {
            "id": f"event_{datetime.now().timestamp()}",
            "title": title,
            "description": description,
            "date": date,
            "time": time,
            "status": "created",
            "created_at": datetime.now().isoformat()
        }
        return {"message": "Event created successfully", "event": event}
    
    elif action == "list":
        # Mock events
        events = [
            {"id": "1", "title": "Team Meeting", "date": "2024-01-15", "time": "10:00"},
            {"id": "2", "title": "Project Review", "date": "2024-01-16", "time": "14:00"}
        ]
        return {"events": events, "count": len(events)}
    
    return {"error": f"Unknown action: {action}"}

# Define business tool specifications
currency_tool = {
    "type": "function",
    "function": {
        "name": "currency_converter",
        "description": "Convert amounts between different currencies",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {"type": "number", "description": "Amount to convert"},
                "from_currency": {"type": "string", "description": "Source currency code (USD, EUR, GBP, etc.)"},
                "to_currency": {"type": "string", "description": "Target currency code"}
            },
            "required": ["amount", "from_currency", "to_currency"]
        }
    }
}

schedule_tool = {
    "type": "function",
    "function": {
        "name": "schedule_manager",
        "description": "Manage calendar events and scheduling",
        "parameters": {
            "type": "object",
            "properties": {
                "action": {"type": "string", "enum": ["create", "list", "delete"], "description": "Action to perform"},
                "date": {"type": "string", "description": "Date in YYYY-MM-DD format"},
                "time": {"type": "string", "description": "Time in HH:MM format"},
                "title": {"type": "string", "description": "Event title"},
                "description": {"type": "string", "description": "Event description"}
            },
            "required": ["action"]
        }
    }
}

# Business tool suite
business_tools = [currency_tool, schedule_tool, calculator_tool, time_tool]
business_functions = {
    "currency_converter": currency_converter,
    "schedule_manager": schedule_manager,
    "calculate": calculate,
    "get_time_info": get_time_info
}

print("💼 Business tool suite ready!")

💼 Business tool suite ready!


In [13]:
# Test business scenario
business_query = """
I'm planning a business trip to Europe. Can you help me with:
1. Convert $2,500 USD to EUR
2. Convert £1,200 GBP to EUR
3. Calculate the total in EUR
4. Create a calendar event for "Business Trip to Paris" on 2024-02-15 at 09:00
5. Show me my current events
"""

print("💼 Processing business scenario...\n")

messages, response = robust_tool_calling_conversation(
    business_query,
    business_tools,
    business_functions
)

💼 Processing business scenario...


🔄 Iteration 1
🤖 AI Response: <think>
This is a multi-part request that requires several actions. Let me break it down:
1. Convert USD to EUR ($2,500)
2. Convert GBP to EUR (£1,200)
3. Calculate total in EUR
4. Create calendar event for Paris trip
5. Show current events

For parts 1-3, I need to use the currency_converter function twice, then perform an addition. For parts 4-5, I need to use the schedule_manager function.
</think>


I'll help you with your business trip planning. Let me handle each request step by step.

First, I need to convert your currencies to EUR:

🔧 Executing: currency_converter
📝 Arguments: {'amount': 2500, 'from_currency': 'USD', 'to_currency': 'EUR'}
✅ Success: {'original_amount': 2500, 'converted_amount': 2125.0, 'from_currency': 'USD', 'to_currency': 'EUR', 'exchange_rate': 0.85, 'timestamp': '2025-07-17T13:34:31.621035'}
🔧 Executing: currency_converter
📝 Arguments: {'amount': 1200, 'from_currency': 'GBP', 'to_currency': 'E

## 🏆 Best Practices and Tips

Here are some important best practices when working with tool calling in HelpingAI:

### 🎯 Tool Design Best Practices

1. **Clear Function Names**: Use descriptive, action-oriented names
2. **Detailed Descriptions**: Provide comprehensive descriptions of what each tool does
3. **Proper Parameter Types**: Use appropriate JSON schema types
4. **Required vs Optional**: Clearly mark required parameters
5. **Error Handling**: Always include robust error handling
6. **Validation**: Validate inputs before processing
7. **Timeouts**: Implement timeouts for long-running operations
8. **Logging**: Log tool executions for debugging

### 🔄 Conversation Flow

1. **Message History**: Always maintain complete message history
2. **Tool Results**: Include tool results in the conversation
3. **Error Recovery**: Handle tool errors gracefully
4. **Iteration Limits**: Prevent infinite loops with max iterations
5. **Context Preservation**: Keep important context throughout the conversation

### 🛡️ Security Considerations

1. **Input Sanitization**: Sanitize all tool inputs
2. **Permission Checks**: Verify user permissions before executing tools
3. **Rate Limiting**: Implement rate limiting for tool calls
4. **Audit Logging**: Log all tool executions for security auditing
5. **Sensitive Data**: Never log sensitive information

In [14]:
# Example of a well-designed, secure tool
import hashlib
import time

class SecureToolExecutor:
    def __init__(self):
        self.call_history = []
        self.rate_limits = {}  # user_id -> list of timestamps
    
    def rate_limit_check(self, user_id, max_calls=10, time_window=60):
        """Check if user is within rate limits"""
        now = time.time()
        if user_id not in self.rate_limits:
            self.rate_limits[user_id] = []
        
        # Clean old timestamps
        self.rate_limits[user_id] = [
            ts for ts in self.rate_limits[user_id] 
            if now - ts < time_window
        ]
        
        # Check limit
        if len(self.rate_limits[user_id]) >= max_calls:
            return False
        
        # Add current call
        self.rate_limits[user_id].append(now)
        return True
    
    def secure_file_hash(self, filepath, user_id="demo_user"):
        """Securely calculate file hash with proper validation"""
        # Rate limiting
        if not self.rate_limit_check(user_id):
            return {"error": "Rate limit exceeded. Try again later."}
        
        # Input validation
        if not filepath or not isinstance(filepath, str):
            return {"error": "Invalid filepath provided"}
        
        # Security check - prevent path traversal
        if ".." in filepath or filepath.startswith("/"):
            return {"error": "Invalid filepath: path traversal detected"}
        
        # Log the attempt (without sensitive data)
        attempt_hash = hashlib.sha256(f"{user_id}:{filepath}:{time.time()}".encode()).hexdigest()[:8]
        self.call_history.append({
            "timestamp": time.time(),
            "user_id_hash": hashlib.sha256(user_id.encode()).hexdigest()[:8],
            "attempt_id": attempt_hash,
            "tool": "secure_file_hash"
        })
        
        try:
            # Mock implementation (in real use, implement actual file reading)
            mock_hash = hashlib.sha256(filepath.encode()).hexdigest()
            return {
                "filepath": filepath,
                "hash": mock_hash,
                "algorithm": "SHA-256",
                "timestamp": datetime.now().isoformat(),
                "attempt_id": attempt_hash
            }
        except Exception as e:
            return {"error": f"Processing failed: {type(e).__name__}"}

# Initialize secure executor
secure_executor = SecureToolExecutor()

# Test the secure tool
print("🔒 Testing secure tool implementation:")
print("✅ Valid request:", secure_executor.secure_file_hash("document.pdf"))
print("❌ Invalid request:", secure_executor.secure_file_hash("../../../etc/passwd"))
print(f"📊 Call history: {len(secure_executor.call_history)} calls logged")

🔒 Testing secure tool implementation:
✅ Valid request: {'filepath': 'document.pdf', 'hash': '5cf35f3f6ff897f2bdde5a25a9616223ae2327746b723e3691060fa610780815', 'algorithm': 'SHA-256', 'timestamp': '2025-07-17T13:34:38.345609', 'attempt_id': '7ae356f2'}
❌ Invalid request: {'error': 'Invalid filepath: path traversal detected'}
📊 Call history: 1 calls logged


## 🎓 Conclusion

You've now learned how to implement comprehensive tool calling with HelpingAI! Here's what we covered:

### ✅ What You Learned
- **Basic tool definition** and execution
- **Multi-tool environments** with intelligent tool selection
- **Streaming responses** with tool calling
- **Error handling** and robust execution
- **Business-specific tools** for real-world scenarios
- **Security best practices** for production use

### 🚀 Next Steps
1. **Experiment** with your own custom tools
2. **Integrate** with your existing APIs and services
3. **Build** complete applications using tool calling
4. **Implement** proper security and monitoring
5. **Scale** your tool ecosystem as needed


**Happy coding with HelpingAI! 🎉**